In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('/CIMA/Data/ALFA_freq/match_MAF_20250320.csv',index_col=0)

In [4]:
df = df[['ID','REF','ALT','EUR','AFR','TOT']]

In [5]:
# 节选 ALT 列中含逗号的行
df_with_comma = df[df['ALT'].str.contains(',')]

# 节选 ALT 列中不含逗号的行
df_without_comma = df[~df['ALT'].str.contains(',')]

In [6]:
#展开数据框
def expand_row(row):
    alts = row['ALT'].split(',')
    eur_counts = row['EUR'].split(':')[1].split(',')
    afr_counts = row['AFR'].split(':')[1].split(',')
    tot_counts = row['TOT'].split(':')[1].split(',')
    
    expanded_rows = []
    for i, alt in enumerate(alts):
        expanded_rows.append({
            'ID': row['ID'],
            'REF': row['REF'],
            'ALT': alt,
            'EUR': f"{row['EUR'].split(':')[0]}:{eur_counts[i]}",
            'AFR': f"{row['AFR'].split(':')[0]}:{afr_counts[i]}",
            'TOT': f"{row['TOT'].split(':')[0]}:{tot_counts[i]}"
        })
    return expanded_rows

# 展开数据框
expanded_data = df_with_comma.apply(expand_row, axis=1).explode()
df_with_comma = pd.DataFrame(expanded_data.tolist())

In [7]:
df_with_comma

,ID,REF,ALT,EUR,AFR,TOT
0,rs7843844,T,A,14020:0,2756:0,18430:0
1,rs7843844,T,C,14020:2853,2756:479,18430:3757
2,rs4045957,G,A,79300:0,3358:0,93774:0
3,rs4045957,G,T,79300:47720,3358:2049,93774:56491
4,rs60319211,C,A,13998:0,2750:0,18270:0
...,...,...,...,...,...,...
104880,rs11223791,T,G,11044:0,2452:0,14704:0
104881,rs7122711,T,A,14278:0,2448:0,18678:0
104882,rs7122711,T,C,14278:13067,2448:2363,18678:17187
104883,rs60471450,G,A,13630:0,2502:0,17558:0


In [8]:
df_all = pd.concat([df_with_comma,df_without_comma])

In [9]:
df_all['pair'] = df_all['ID']+'_'+df_all['REF']+'_'+df_all['ALT']

In [10]:
df_all

,ID,REF,ALT,EUR,AFR,TOT,pair
0,rs7843844,T,A,14020:0,2756:0,18430:0,rs7843844_T_A
1,rs7843844,T,C,14020:2853,2756:479,18430:3757,rs7843844_T_C
2,rs4045957,G,A,79300:0,3358:0,93774:0,rs4045957_G_A
3,rs4045957,G,T,79300:47720,3358:2049,93774:56491,rs4045957_G_T
4,rs60319211,C,A,13998:0,2750:0,18270:0,rs60319211_C_A
...,...,...,...,...,...,...,...
112821,rs4937915,A,G,24664:14371,3168:2633,30780:18780,rs4937915_A_G
112823,rs12799211,C,T,15802:2157,3248:166,27274:5195,rs12799211_C_T
112824,rs4245137,A,G,14284:13092,2946:2712,18888:17285,rs4245137_A_G
112825,rs11223926,G,A,35500:1512,5608:438,53428:2800,rs11223926_G_A


In [11]:
CIMA_loci = pd.read_csv('/CIMA/Data/ALFA_freq/20250321_lead_qtl.avinput.hg38_avsnp150_dropped',sep=r'\s+',header=None)

In [12]:
CIMA_loci['pair'] = CIMA_loci[1]+'_'+CIMA_loci[5]+'_'+CIMA_loci[6]

In [13]:
df_all = df_all[df_all['pair'].isin(CIMA_loci['pair'])]

In [14]:
def convert_ratio(value):
    try:
        ref, alt = map(float, value.split(':'))
        return alt / ref
    except (ValueError, ZeroDivisionError):
        return None

In [15]:
df_all['EUR'] = df_all['EUR'].apply(convert_ratio)
df_all['AFR'] = df_all['AFR'].apply(convert_ratio)
df_all['TOT'] = df_all['TOT'].apply(convert_ratio)

/tmp/ipykernel_2946958/2267949795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['EUR'] = df_all['EUR'].apply(convert_ratio)
/tmp/ipykernel_2946958/2267949795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['AFR'] = df_all['AFR'].apply(convert_ratio)
/tmp/ipykernel_2946958/2267949795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [16]:
df_all

,ID,REF,ALT,EUR,AFR,TOT,pair
1,rs7843844,T,C,0.203495,0.173803,0.203852,rs7843844_T_C
3,rs4045957,G,T,0.601765,0.610185,0.602416,rs4045957_G_T
6,rs60319211,C,T,0.008073,0.001091,0.006623,rs60319211_C_T
7,rs12549433,G,A,0.149084,0.561909,0.216435,rs12549433_G_A
10,rs201628711,T,C,0.061471,0.000000,0.054102,rs201628711_T_C
...,...,...,...,...,...,...,...
112821,rs4937915,A,G,0.582671,0.831124,0.610136,rs4937915_A_G
112823,rs12799211,C,T,0.136502,0.051108,0.190474,rs12799211_C_T
112824,rs4245137,A,G,0.916550,0.920570,0.915131,rs4245137_A_G
112825,rs11223926,G,A,0.042592,0.078103,0.052407,rs11223926_G_A


In [17]:
for col in ['EUR', 'AFR', 'TOT']:
    df_all[col] = df_all[col].apply(lambda x: 1 - x if x > 0.5 else x)

/tmp/ipykernel_2946958/778234432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[col] = df_all[col].apply(lambda x: 1 - x if x > 0.5 else x)


In [18]:
df_filtered = df_all[(df_all['EUR'] < 0.01) | (df_all['AFR'] < 0.01) | (df_all['TOT'] < 0.01)]

In [19]:
df_filtered

,ID,REF,ALT,EUR,AFR,TOT,pair
6,rs60319211,C,T,0.008073,0.001091,0.006623,rs60319211_C_T
10,rs201628711,T,C,0.061471,0.000000,0.054102,rs201628711_T_C
15,rs13249267,C,G,0.317530,0.000000,0.249581,rs13249267_C_G
17,rs34905243,A,C,0.057115,0.003791,0.046617,rs34905243_A_C
27,rs75661751,C,G,0.044944,0.004277,0.036976,rs75661751_C_G
...,...,...,...,...,...,...,...
112677,rs76979670,C,T,0.002387,0.085597,0.013681,rs76979670_C_T
112694,rs73583468,A,G,0.003010,0.054650,0.013976,rs73583468_A_G
112712,rs11221987,C,T,0.036749,0.008147,0.031869,rs11221987_C_T
112805,rs574608,C,T,0.030239,0.008147,0.028745,rs574608_C_T


In [20]:
CIMA_loci['variant_id'] = CIMA_loci[2]+'_'+CIMA_loci[3].astype(str)

In [21]:
CIMA_loci = CIMA_loci[['pair','variant_id']]

In [22]:
df_filtered = pd.merge(df_filtered,CIMA_loci,on='pair')

In [23]:
df_filtered

,ID,REF,ALT,EUR,AFR,TOT,pair,variant_id
0,rs60319211,C,T,0.008073,0.001091,0.006623,rs60319211_C_T,chr8_286615
1,rs201628711,T,C,0.061471,0.000000,0.054102,rs201628711_T_C,chr8_360236
2,rs13249267,C,G,0.317530,0.000000,0.249581,rs13249267_C_G,chr8_386881
3,rs34905243,A,C,0.057115,0.003791,0.046617,rs34905243_A_C,chr8_418976
4,rs75661751,C,G,0.044944,0.004277,0.036976,rs75661751_C_G,chr8_565510
...,...,...,...,...,...,...,...,...
13171,rs76979670,C,T,0.002387,0.085597,0.013681,rs76979670_C_T,chr11_130059642
13172,rs73583468,A,G,0.003010,0.054650,0.013976,rs73583468_A_G,chr11_130089937
13173,rs11221987,C,T,0.036749,0.008147,0.031869,rs11221987_C_T,chr11_130166963
13174,rs574608,C,T,0.030239,0.008147,0.028745,rs574608_C_T,chr11_134240906


In [24]:
df_all = pd.merge(df_all,CIMA_loci,on='pair')

In [25]:
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')

genotype_df = genotype_df.loc[df_filtered['variant_id'],]

genotype_df = genotype_df.transpose()

CIMA_AF = genotype_df.sum()/826

CIMA_AF = CIMA_AF.reset_index()

CIMA_AF.columns = ['variant_id','CIMA']

df_filtered = pd.merge(df_filtered,CIMA_AF,on='variant_id')

df_filtered['CIMA'] = df_filtered['CIMA'].apply(lambda x: 1 - x if x > 0.5 else x)

In [26]:
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')

genotype_df = genotype_df.loc[df_all['variant_id'],]

genotype_df = genotype_df.transpose()

CIMA_AF = genotype_df.sum()/826

CIMA_AF = CIMA_AF.reset_index()

CIMA_AF.columns = ['variant_id','CIMA']

df_all = pd.merge(df_all,CIMA_AF,on='variant_id')

df_all['CIMA'] = df_all['CIMA'].apply(lambda x: 1 - x if x > 0.5 else x)

In [27]:
df_all

,ID,REF,ALT,EUR,AFR,TOT,pair,variant_id,CIMA
0,rs7843844,T,C,0.203495,0.173803,0.203852,rs7843844_T_C,chr8_253429,0.412833
1,rs4045957,G,T,0.398235,0.389815,0.397584,rs4045957_G_T,chr8_269868,0.406780
2,rs60319211,C,T,0.008073,0.001091,0.006623,rs60319211_C_T,chr8_286615,0.279661
3,rs12549433,G,A,0.149084,0.438091,0.216435,rs12549433_G_A,chr8_359027,0.337772
4,rs201628711,T,C,0.061471,0.000000,0.054102,rs201628711_T_C,chr8_360236,0.336562
...,...,...,...,...,...,...,...,...,...
109072,rs4937915,A,G,0.417329,0.168876,0.389864,rs4937915_A_G,chr11_134576576,0.432203
109073,rs12799211,C,T,0.136502,0.051108,0.190474,rs12799211_C_T,chr11_134587705,0.466102
109074,rs4245137,A,G,0.083450,0.079430,0.084869,rs4245137_A_G,chr11_134588224,0.323245
109075,rs11223926,G,A,0.042592,0.078103,0.052407,rs11223926_G_A,chr11_134647584,0.330508


In [28]:
df_all.to_csv('/CIMA/Data/ALFA_freq/20250320_AFR_EUR_TOT_CIMA_all.csv')

In [118]:
df_filtered.to_csv('/CIMA/Data/ALFA_freq/20250320_AFR_EUR_TOT_CIMA.csv')

In [112]:
SMR_result = pd.read_csv('/CIMA/Result/downstream/SMR_summary/merge_all_SMR_result.csv',index_col=0)

In [115]:
pd.merge(SMR_result,df_filtered,left_on='topSNP',right_on='variant_id').to_csv('/CIMA/Result/downstream/SMR_summary/merge_all_SMR_result_EAS_specific.csv')